In [599]:
import pandas as pd

columns_to_import = [
    'Search Frequency Rank', 'Search Term', 'Top Clicked Product #1: ASIN',
    'Top Clicked Product #1: Product Title', 'Top Clicked Product #1: Click Share',
    'Top Clicked Product #1: Conversion Share', 'Top Clicked Product #2: ASIN',
    'Top Clicked Product #2: Product Title', 'Top Clicked Product #2: Click Share',
    'Top Clicked Product #2: Conversion Share', 'Top Clicked Product #3: ASIN',
    'Top Clicked Product #3: Product Title', 'Top Clicked Product #3: Click Share',
    'Top Clicked Product #3: Conversion Share'
]

# Функція для фільтрації рядків
def filter_chunks(chunk):
    return chunk[chunk['Search Term'].str.contains('the substance', na=False)]

# Імпорт та фільтрація даних
chunks = pd.read_csv(
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_02.csv',
    skiprows=1,
    usecols=columns_to_import,
    chunksize=10000  # Розмір частини може бути налаштований
)

filtered_df = pd.concat([filter_chunks(chunk) for chunk in chunks])

filtered_df.head()


,Search Frequency Rank,Search Term,Top Clicked Product #1: ASIN,Top Clicked Product #1: Product Title,Top Clicked Product #1: Click Share,Top Clicked Product #1: Conversion Share,Top Clicked Product #2: ASIN,Top Clicked Product #2: Product Title,Top Clicked Product #2: Click Share,Top Clicked Product #2: Conversion Share,Top Clicked Product #3: ASIN,Top Clicked Product #3: Product Title,Top Clicked Product #3: Click Share,Top Clicked Product #3: Conversion Share
110,111,the substance,B0D72D8BR8,The Substance,88.05,0.00,B0D72B2HLW,The Substance,8.14,0.0,B0DMWTK3N8,The Substance [Blu-Ray],1.40,31.82
7308,7301,the substance 2024,B0D72D8BR8,The Substance,87.87,NaN,B0D72B2HLW,The Substance,9.83,NaN,B0DCNS4TSG,Blink Twice,0.84,NaN
44781,44595,the substance 4k,B0DMWRT6ZP,The Substance [4K UHD],89.27,88.46,B0DLCX1N2R,The Cell [Limited Edition] [4K UHD],3.90,0.0,B0D72B2HLW,The Substance,1.46,0.00
90914,90089,the substance blu ray,B0DMWTK3N8,The Substance [Blu-Ray],93.46,100.00,B0DMWRT6ZP,The Substance [4K UHD],3.74,0.0,B0029XFN8A,The Girl Next Door (Unrated Edition) [Blu-ray],0.93,0.00
93811,92969,the substance movie,B0D72D8BR8,The Substance,63.46,NaN,B0D72B2HLW,The Substance,27.88,NaN,B0DMWRT6ZP,The Substance [4K UHD],2.88,NaN


In [78]:
import pandas as pd
from typing import List, Dict, Tuple, Optional
from datetime import datetime, timedelta
from pathlib import Path

pd.set_option('future.no_silent_downcasting', True)

In [2]:
def load_and_filter_by_rank(
    file_path: str,
    columns_to_import: List[str],
    rank_range: Tuple[int, int],
    chunk_size: int = 10000
) -> pd.DataFrame:
    """
    Loads and filters the initial file for search terms within specified SFR (Search Frequency Rank) range.
    
    Parameters:
    -----------
    file_path : str
        Path to the CSV file
    columns_to_import : List[str]
        List of column names to import
    rank_range : Tuple[int, int]
        Tuple of (min_rank, max_rank) to filter by
    chunk_size : int, optional
        Size of chunks for reading large files, default is 10000
        
    Returns:
    --------
    pd.DataFrame
        Filtered DataFrame containing only rows where SFR is within the specified range
    """
    def filter_chunks(chunk: pd.DataFrame) -> pd.DataFrame:
        # Convert SFR to numeric, coerce errors to NaN
        chunk['Search Frequency Rank'] = pd.to_numeric(chunk['Search Frequency Rank'], errors='coerce')
        # Filter by rank range and remove any NaN values
        return chunk[
            (chunk['Search Frequency Rank'] >= rank_range[0]) & 
            (chunk['Search Frequency Rank'] <= rank_range[1])
        ].dropna(subset=['Search Frequency Rank'])
    
    chunks = pd.read_csv(
        file_path,
        skiprows=1,
        usecols=columns_to_import,
        chunksize=chunk_size
    )
    
    filtered_df = pd.concat([filter_chunks(chunk) for chunk in chunks])
    
    # Sort by rank to ensure ordered results
    return filtered_df.sort_values('Search Frequency Rank')



In [618]:


def load_and_filter_initial_file(
    file_path: str,
    columns_to_import: List[str],
    search_term: str,
    chunk_size: int = 10000
) -> pd.DataFrame:
    """
    Loads and filters the initial file for specific search terms.
    """
    def filter_chunks(chunk: pd.DataFrame) -> pd.DataFrame:
        return chunk[chunk['Search Term'].str.contains(search_term, na=False)]
    
    chunks = pd.read_csv(
        file_path,
        skiprows=1,
        usecols=columns_to_import,
        chunksize=chunk_size
    )
    return pd.concat([filter_chunks(chunk) for chunk in chunks])

def load_and_filter_subsequent_files(
    file_paths: List[str],
    columns_to_import: List[str],
    filtered_terms: List[str],
    chunk_size: int = 10000
) -> List[pd.DataFrame]:
    """
    Loads and filters subsequent files based on search terms from initial file.
    """
    def filter_chunks(chunk: pd.DataFrame) -> pd.DataFrame:
        return chunk[chunk['Search Term'].isin(filtered_terms)]
    
    filtered_dfs = []
    for file_path in file_paths:
        chunks = pd.read_csv(
            file_path,
            skiprows=1,
            usecols=columns_to_import,
            chunksize=chunk_size
        )
        filtered_df = pd.concat([filter_chunks(chunk) for chunk in chunks])
        filtered_dfs.append(filtered_df)
    
    return filtered_dfs

def rename_columns(df: pd.DataFrame, column_mapping: Dict[str, str]) -> pd.DataFrame:
    """
    Renames DataFrame columns according to the provided mapping.
    """
    return df.rename(columns=column_mapping)

def find_conversions(row: pd.Series) -> Tuple[Dict[str, int], float]:
    """
    Calculates conversion metrics for a given row.
    """
    row = row.copy()
    row.loc[['Conversion_Share_1', 'Conversion_Share_2', 'Conversion_Share_3']] = (
    row.loc[['Conversion_Share_1', 'Conversion_Share_2', 'Conversion_Share_3']]
    .fillna(0)
    .infer_objects(copy=False)
)
    if (row['Conversion_Share_1'] == 0 and 
        row['Conversion_Share_2'] == 0 and 
        row['Conversion_Share_3'] == 0):
        return {
            'Total_Orders': 0,
            'Other_Orders': 0,
            'ASIN1_Orders': 0,
            'ASIN2_Orders': 0,
            'ASIN3_Orders': 0, 
            'SFR': row['SFR'] if 'SFR' in row else None
        }, 0

    conv_shares = [row['Conversion_Share_1'], 
                  row['Conversion_Share_2'], 
                  row['Conversion_Share_3']]
    min_share = min(share for share in conv_shares if share > 0)
    ratios = [share/min_share if share > 0 else 0 for share in conv_shares]
    delta = 0.025
    start = 1
    if row['SFR'] <= 200 and min_share >= 1:
        start = 2
    if row['SFR'] <= 4000:
        if any(share == 0 for share in conv_shares) and min_share >= 20:
            delta = 0.005
            start = 20
        elif  any(share == 0 for share in conv_shares) and min_share < 2 and min_share >= 1:   
            delta = 0.005
            start =  5
        elif  any(share == 0 for share in conv_shares) and min_share > 2 and min_share < 20:   
            delta = 0.005
            start =  15   
    elif 4000 < row['SFR'] <= 10000:
        if any(share == 0 for share in conv_shares) and min_share >= 2:
            delta = 0.01
            start = 5

    if row['SFR'] <= 100:
        a = 1600
        b = 100
    elif 100 < row['SFR'] <= 500:
        a = 1300
        b = 90
    elif 500 < row['SFR'] <= 1000:
        a = 1000
        b = 80
    elif 1000 < row['SFR'] <= 2000:
        a = 900 
        b = 70
    elif 2000 < row['SFR'] <= 5000:
        a = 800
        b = 60
    elif 5000 < row['SFR'] <= 15000:
        a = 700
        b = 55
    else:
        a = 600
        b = 50
    while delta <= 3:
        for i in range(start, b):
            predicted = [ratio * i for ratio in ratios]
            
            if all(abs(pred - round(pred)) < delta for pred in predicted if pred > 0):
                orders = [round(pred) for pred in predicted]
                sum_ratio = sum(conv_shares)
                other_orders = round((100-sum_ratio)*sum(orders)/sum_ratio) if sum_ratio > 0 else 0
                total_orders = sum(orders) + other_orders
                if total_orders > a and delta < 1.5:
                    break
                if total_orders % 1000 != 0:
                    return {
                        'Total_Orders': total_orders,
                        'Other_Orders': other_orders,
                        'ASIN1_Orders': orders[0],
                        'ASIN2_Orders': orders[1],
                        'ASIN3_Orders': orders[2],
                        'SFR': row['SFR'] if 'SFR' in row else None
                    }, delta
        delta += 0.02 
    return {
        'Total_Orders': 0,
        'Other_Orders': 0,
        'ASIN1_Orders': 0,
        'ASIN2_Orders': 0,
        'ASIN3_Orders': 0,
        'SFR': row['SFR'] if 'SFR' in row else None
    }, delta


def find_clicks(row: pd.Series, conv_row: pd.Series) -> Tuple[Dict[str, int], float]:
    """
    Calculates click metrics for a given row based on conversion data.
    """
    click_shares = [
        row['Click_Share_1'], 
        row['Click_Share_2'], 
        row['Click_Share_3']
    ]
    min_click_share = min(share for share in click_shares if share > 0)
    click_ratios = [share/min_click_share if share > 0 else 0 for share in click_shares]
    delta = 0.025
    if row['SFR'] <= 100:
        a = 500
        b = 2.7
    elif 100 < row['SFR'] <= 500:
        a = 400
        b = 2.4
    elif 500 < row['SFR'] <= 1000:
        a = 300
        b = 2
    elif 1000 < row['SFR'] <= 5000:
        a = 200
        b = 1.7
    elif 5000 < row['SFR'] <= 20000:
        a = 200
        b = 1.4    
    else:
        a = 150
        b = 1
    while delta <= 3:
        for i in range(1, a):
            predicted_clicks = [ratio * i for ratio in click_ratios]
            
            if all(abs(click - round(click)) < delta for click in predicted_clicks if click > 0):
                clicks = [round(click) for click in predicted_clicks]
                sum_ratio = sum(click_shares)
                other_clicks = round((100-sum_ratio)*sum(clicks)/sum_ratio) if sum_ratio > 0 else 0
                total_clicks = sum(clicks) + other_clicks
                
                if (total_clicks % 1000 != 0 and
                    total_clicks >= conv_row['Total_Orders'] * b and
                    other_clicks >= conv_row['Other_Orders'] and
                    all(c >= o for c, o in zip(clicks, [
                        conv_row['ASIN1_Orders'],
                        conv_row['ASIN2_Orders'],
                        conv_row['ASIN3_Orders']
                    ]))):
                    
                    return {
                        'Total_Clicks': total_clicks,
                        'Other_Clicks': other_clicks,
                        'ASIN1_Clicks': clicks[0],
                        'ASIN2_Clicks': clicks[1],
                        'ASIN3_Clicks': clicks[2]
                    }, delta
        delta += 0.02
        if delta > 2.8:
            a += 50      
    return {
        'Total_Clicks': 1,
        'Other_Clicks': 1,
        'ASIN1_Clicks': 0,
        'ASIN2_Clicks': 0,
        'ASIN3_Clicks': 0
    }, delta

    
def process_search_term(
    search_term: str,
    all_dfs: List[pd.DataFrame],
    dates: List[datetime]
) -> pd.DataFrame:
    """
    Processes a single search term across all DataFrames and returns results.
    """
    conversion_results = []
    click_results = []
    
    for df, date in zip(all_dfs, dates):
        row = df[df['Search_Term'] == search_term]
        if row.empty:
            conv_result = {
                'Total_Orders': 0,
                'Other_Orders': 0,
                'ASIN1_Orders': 0,
                'ASIN2_Orders': 0,
                'ASIN3_Orders': 0,
                'SFR': None
            }
            click_result = {
                'Total_Clicks': 0,
                'Other_Clicks': 0,
                'ASIN1_Clicks': 0,
                'ASIN2_Clicks': 0,
                'ASIN3_Clicks': 0
            }
        else:
            conv_result, _ = find_conversions(row.iloc[0])
            click_result, _ = find_clicks(row.iloc[0], pd.Series(conv_result))
        
        conversion_results.append(conv_result)
        click_results.append(click_result)
    
    conv_df = pd.DataFrame(conversion_results, index=dates)
    
    click_df = pd.DataFrame(click_results, index=dates)
    conv_df['Search_Term'] = search_term
    result_df = pd.concat([conv_df, click_df], axis=1)
    cols = ['Search_Term', 'SFR'] + [col for col in result_df.columns if col not in ['Search_Term', 'SFR']]
    result_df = result_df[cols]
    
    return result_df

    
def analyze_search_terms(
    initial_file: str,
    subsequent_files: List[str],
    columns_to_import: List[str],
    search_term_filter: Optional[str],
    rank_range: Optional[Tuple[int, int]],
    column_mapping: Dict[str, str],
    start_date: str
) -> List[pd.DataFrame]:
    """
    Main function to analyze search terms across multiple files.
    """
    # Load and process initial file
    if search_term_filter is None:
        # Load and process initial file using load_and_filter_by_rank
        initial_df = load_and_filter_by_rank(
            initial_file,
            columns_to_import,
            rank_range
        )
    else:
        # Load and process initial file using load_and_filter_initial_file
        initial_df = load_and_filter_initial_file(
            initial_file,
            columns_to_import,
            search_term_filter
        )
    # Get unique search terms
    filtered_terms = initial_df['Search Term'].unique()
    
    # Load and process subsequent files
    all_dfs = [initial_df] + load_and_filter_subsequent_files(
        subsequent_files,
        columns_to_import,
        filtered_terms
    )
    
    # Rename columns in all DataFrames
    all_dfs = [rename_columns(df, column_mapping) for df in all_dfs]
    
    # Generate dates
    dates = pd.date_range(start=start_date, freq='D', periods=len(all_dfs))
    
    # Process each search term
    result_dfs = []
    for search_term in filtered_terms:
        result_df = process_search_term(search_term, all_dfs, dates)
        result_dfs.append(result_df)
    
    return result_dfs



In [619]:
columns_to_import = [
        'Search Frequency Rank', 'Search Term', 'Top Clicked Product #1: ASIN',
        'Top Clicked Product #1: Product Title', 'Top Clicked Product #1: Click Share',
        'Top Clicked Product #1: Conversion Share', 'Top Clicked Product #2: ASIN',
        'Top Clicked Product #2: Product Title', 'Top Clicked Product #2: Click Share',
        'Top Clicked Product #2: Conversion Share', 'Top Clicked Product #3: ASIN',
        'Top Clicked Product #3: Product Title', 'Top Clicked Product #3: Click Share',
        'Top Clicked Product #3: Conversion Share'
    ]
    
column_mapping = {
    'Search Frequency Rank': 'SFR',
    'Search Term': 'Search_Term',
    'Top Clicked Product #1: ASIN': 'ASIN_1',
    'Top Clicked Product #1: Product Title': 'Title_1',
    'Top Clicked Product #1: Click Share': 'Click_Share_1',
    'Top Clicked Product #1: Conversion Share': 'Conversion_Share_1',
    'Top Clicked Product #2: ASIN': 'ASIN_2',
    'Top Clicked Product #2: Product Title': 'Title_2',
    'Top Clicked Product #2: Click Share': 'Click_Share_2',
    'Top Clicked Product #2: Conversion Share': 'Conversion_Share_2',
    'Top Clicked Product #3: ASIN': 'ASIN_3',
    'Top Clicked Product #3: Product Title': 'Title_3',
    'Top Clicked Product #3: Click Share': 'Click_Share_3',
    'Top Clicked Product #3: Conversion Share': 'Conversion_Share_3'
}

initial_file = r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_01.csv'

subsequent_files = [
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_02.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_03.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_04.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_05.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_06.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_07.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_08.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_09.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_10.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_11.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_12.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_13.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_14.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_15.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_16.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_17.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_18.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_19.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_20.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_21.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_22.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_23.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_24.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_25.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_26.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_27.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_28.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_29.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_30.csv',
    r'C:\Users\user\Downloads\US_Top_Search_Terms_Simple_Day_2025_01_31.csv'
    
]



In [621]:
results_3[93].tail(10)

,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-22,neck wrinkle patches,1674,65,45,0,0,20,8039,121,3998,3879,41
2025-01-23,neck wrinkle patches,1507,60,40,0,0,20,4651,59,2425,2147,20
2025-01-24,neck wrinkle patches,275,0,0,0,0,0,1875,11,932,929,3
2025-01-25,neck wrinkle patches,785,90,75,0,0,15,8776,96,4566,4071,43
2025-01-26,neck wrinkle patches,1693,90,75,0,0,15,11750,174,5775,5754,47
2025-01-27,neck wrinkle patches,455,210,195,0,0,15,28334,247,14331,13688,68
2025-01-28,neck wrinkle patches,249,90,75,0,0,15,14762,84,7762,6885,31
2025-01-29,neck wrinkle patches,395,150,135,0,0,15,27692,152,13813,13655,72
2025-01-30,neck wrinkle patches,577,0,0,0,0,0,690,8,345,335,2
2025-01-31,neck wrinkle patches,263,100,80,0,0,20,17500,103,9163,8206,28


In [465]:
results = analyze_search_terms(
    initial_file,
    subsequent_files,
    columns_to_import,
    'puzzles for kids',
    None, 
    column_mapping,
    '2025-01-01'
)
len(results)

19

In [467]:
results[0].tail()

,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-27,puzzles for kids ages 4-6,17297,79,66,1,8,4,399,309,40,29,21
2025-01-28,puzzles for kids ages 4-6,16156,23,19,1,1,2,409,319,32,31,27
2025-01-29,puzzles for kids ages 4-6,15167,88,83,1,3,1,428,336,34,31,27
2025-01-30,puzzles for kids ages 4-6,18134,23,20,1,1,1,365,301,24,21,19
2025-01-31,puzzles for kids ages 4-6,16752,98,85,3,5,5,394,323,25,24,22


In [620]:
start_date = '2025-01-01'
results_3 = analyze_search_terms(
    initial_file,
    subsequent_files,
    columns_to_import,
    None,
    (20, 120),
    column_mapping,
    start_date
)
len(results_3)

101

In [622]:
results_3[0].tail()

,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-27,airpods,17,1048,535,461,24,28,2993,1274,1358,188,173
2025-01-28,airpods,26,673,290,262,108,13,3333,1268,1388,476,201
2025-01-29,airpods,18,1177,480,392,283,22,3701,1324,1361,788,228
2025-01-30,airpods,13,820,348,267,190,15,3150,1145,1154,651,200
2025-01-31,airpods,25,524,205,177,131,11,4507,1619,1627,964,297


In [623]:
results_3[88].tail()

,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-27,rice cooker,106,221,147,44,23,7,4701,3095,913,386,307
2025-01-28,rice cooker,133,694,463,134,73,24,2076,1356,399,190,131
2025-01-29,rice cooker,132,935,655,149,115,16,2662,1756,463,287,156
2025-01-30,rice cooker,138,354,225,58,53,18,2287,1491,359,268,169
2025-01-31,rice cooker,186,629,408,93,99,29,1789,1179,266,210,134


In [624]:
results_3[97].head()


,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-01,gel nail polish,117,1131,912,158,47,14,4090,3032,737,166,155
2025-01-02,gel nail polish,135,253,205,37,2,9,4495,3294,832,191,178
2025-01-03,gel nail polish,112,1099,918,153,10,18,3277,2409,618,134,116
2025-01-04,gel nail polish,109,148,122,19,1,6,1845,1370,318,81,76
2025-01-05,gel nail polish,119,220,176,29,12,3,1211,898,204,55,54


In [625]:
results_3[9].tail(10)

,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-22,nad supplement,18,42,22,0,0,20,1544,330,677,349,188
2025-01-23,nad supplement,10,45,25,0,0,20,3621,1052,1448,760,361
2025-01-24,nad supplement,13,0,0,0,0,0,2403,828,629,535,411
2025-01-25,nad supplement,62,834,494,1,339,0,2785,1209,566,549,461
2025-01-26,nad supplement,72,47,27,20,0,0,1373,679,289,204,201
2025-01-27,nad supplement,32,769,768,0,0,1,2308,1006,466,433,403
2025-01-28,nad supplement,25,0,0,0,0,0,1935,715,427,408,385
2025-01-29,nad supplement,29,769,768,1,0,0,2079,850,430,405,394
2025-01-30,nad supplement,40,0,0,0,0,0,2394,1064,462,445,423
2025-01-31,nad supplement,31,769,768,1,0,0,2134,802,454,449,429


In [565]:
import pandas as pd
import numpy as np
from typing import List, Tuple

# корекція даних щодо аномалій 


def adjust_clicks_and_orders(results: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Adjusts anomalous clicks and orders in each DataFrame from the results list.
    """
    adjusted_results = []
    for df in results:
        # Create a copy of the DataFrame to avoid modifying the original
        adjusted_df = df.copy()
        # First adjust clicks
        adjusted_df = adjust_clicks(adjusted_df)
        # Then adjust orders based on adjusted clicks
        adjusted_df = adjust_orders(adjusted_df)
        adjusted_results.append(adjusted_df)
    return adjusted_results


def get_valid_median(series: pd.Series, index) -> float:
    """
    Calculate median excluding zeros and NaN values.
    """
    valid_values = series[(series > 0) & (series <= index)].dropna()
    return valid_values.median() if not valid_values.empty else 0

def get_valid_median_clicks(series: pd.Series) -> float:
    """
    Calculate median excluding zeros and NaN values.
    """
    valid_values = series[series > 0].dropna()
    return valid_values.median() if not valid_values.empty else 0

    
def adjust_clicks(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adjust anomalous click values.
    """
    median_clicks = get_valid_median_clicks(df['Total_Clicks'])
    if median_clicks == 0:
        return df
    lower_bound = 0.7 * median_clicks
    upper_bound = 1.5 * median_clicks
    target_lower = 0.7 * median_clicks
    target_upper = 1.35 * median_clicks
    for idx in df.index:
        clicks = df.at[idx, 'Total_Clicks']
        if clicks == 0:
            continue
        if clicks == 1:
            df.at[idx, 'Total_Clicks'] = round(median_clicks)
        if clicks < lower_bound:
            # Try multiplying
            for multiplier in range(2, 51):
                new_clicks = clicks * multiplier
                if target_lower <= new_clicks <= target_upper:
                    ratio = multiplier
                    df.at[idx, 'Total_Clicks'] = new_clicks
                    df.at[idx, 'Other_Clicks'] = df.at[idx, 'Other_Clicks'] * ratio
                    df.at[idx, 'ASIN1_Clicks'] = df.at[idx, 'ASIN1_Clicks'] * ratio
                    df.at[idx, 'ASIN2_Clicks'] = df.at[idx, 'ASIN2_Clicks'] * ratio
                    df.at[idx, 'ASIN3_Clicks'] = df.at[idx, 'ASIN3_Clicks'] * ratio
                    break    
        elif clicks > upper_bound:
            # Try dividing
            for divisor in range(2, 51):
                new_clicks = clicks / divisor
                if target_lower <= new_clicks <= target_upper:
                    ratio = 1/divisor
                    df.at[idx, 'Total_Clicks'] = round(new_clicks)
                    df.at[idx, 'Other_Clicks'] = round(df.at[idx, 'Other_Clicks'] * ratio)
                    df.at[idx, 'ASIN1_Clicks'] = round(df.at[idx, 'ASIN1_Clicks'] * ratio)
                    df.at[idx, 'ASIN2_Clicks'] = round(df.at[idx, 'ASIN2_Clicks'] * ratio)
                    df.at[idx, 'ASIN3_Clicks'] = round(df.at[idx, 'ASIN3_Clicks'] * ratio)
                    break
    
    return df

def adjust_orders(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adjust anomalous order values while maintaining relationship with clicks.
    """
    sfr = round(df['SFR'].mean())
    if sfr <= 100:
        index = 1700
    elif 100 < sfr <=500:
        index = 1400
    elif 500 < sfr <=1500:
        index = 1200
    elif 1500 < sfr <= 5000:
        index = 1000
    elif 5000 < sfr <= 15000:
        index = 800
    elif 15000 < sfr < 45000:
        index = 600
    else:
        index = 450
    median_orders = get_valid_median(df['Total_Orders'], index)
    if median_orders == 0:
        return df
    lower_bound = 0.6 * median_orders
    upper_bound = 1.4 * median_orders
    target_lower = 0.7 * median_orders
    target_upper = 1.35 * median_orders
    for idx in df.index:
        orders = df.at[idx, 'Total_Orders']
        clicks = df.at[idx, 'Total_Clicks']
        if orders == 0 or clicks == 0:
            continue
        if orders < lower_bound:
            # Try multiplying
            for multiplier in range(2, 51):
                new_orders = orders * multiplier
                if (target_lower <= new_orders <= target_upper and 
                    new_orders <= clicks):
                    ratio = multiplier
                    df.at[idx, 'Total_Orders'] = new_orders
                    df.at[idx, 'Other_Orders'] = df.at[idx, 'Other_Orders'] * ratio
                    df.at[idx, 'ASIN1_Orders'] = df.at[idx, 'ASIN1_Orders'] * ratio
                    df.at[idx, 'ASIN2_Orders'] = df.at[idx, 'ASIN2_Orders'] * ratio
                    df.at[idx, 'ASIN3_Orders'] = df.at[idx, 'ASIN3_Orders'] * ratio
                    break    
        elif orders > upper_bound:
            # Try dividing
            for divisor in range(2, 51):
                new_orders = orders / divisor
                if (target_lower <= new_orders <= target_upper and 
                    new_orders <= clicks):
                    ratio = 1/divisor
                    df.at[idx, 'Total_Orders'] = round(new_orders)
                    df.at[idx, 'Other_Orders'] = round(df.at[idx, 'Other_Orders'] * ratio)
                    df.at[idx, 'ASIN1_Orders'] = round(df.at[idx, 'ASIN1_Orders'] * ratio)
                    df.at[idx, 'ASIN2_Orders'] = round(df.at[idx, 'ASIN2_Orders'] * ratio)
                    df.at[idx, 'ASIN3_Orders'] = round(df.at[idx, 'ASIN3_Orders'] * ratio)
                    break
    
    return df



In [626]:
adjusted_results = adjust_clicks_and_orders(results_3)
adjusted_results[9].tail(10)

,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-22,nad supplement,18,252,132,0,0,120,3088,660,1354,698,376
2025-01-23,nad supplement,10,270,150,0,0,120,1810,526,724,380,180
2025-01-24,nad supplement,13,0,0,0,0,0,2403,828,629,535,411
2025-01-25,nad supplement,62,417,247,0,170,0,2785,1209,566,549,461
2025-01-26,nad supplement,72,282,162,120,0,0,2746,1358,578,408,402
2025-01-27,nad supplement,32,384,384,0,0,0,2308,1006,466,433,403
2025-01-28,nad supplement,25,0,0,0,0,0,1935,715,427,408,385
2025-01-29,nad supplement,29,384,384,0,0,0,2079,850,430,405,394
2025-01-30,nad supplement,40,0,0,0,0,0,2394,1064,462,445,423
2025-01-31,nad supplement,31,384,384,0,0,0,2134,802,454,449,429


In [627]:
adjusted_results = adjust_clicks_and_orders(adjusted_results)
adjusted_results[9].tail(10)

,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-22,nad supplement,18,252,132,0,0,120,3088,660,1354,698,376
2025-01-23,nad supplement,10,270,150,0,0,120,1810,526,724,380,180
2025-01-24,nad supplement,13,0,0,0,0,0,2403,828,629,535,411
2025-01-25,nad supplement,62,417,247,0,170,0,2785,1209,566,549,461
2025-01-26,nad supplement,72,282,162,120,0,0,2746,1358,578,408,402
2025-01-27,nad supplement,32,384,384,0,0,0,2308,1006,466,433,403
2025-01-28,nad supplement,25,0,0,0,0,0,1935,715,427,408,385
2025-01-29,nad supplement,29,384,384,0,0,0,2079,850,430,405,394
2025-01-30,nad supplement,40,0,0,0,0,0,2394,1064,462,445,423
2025-01-31,nad supplement,31,384,384,0,0,0,2134,802,454,449,429


In [644]:
adjusted_results[83].head()

,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-01,instant face lift cream,103,0,0,0,0,0,1028,230,383,258,157
2025-01-02,instant face lift cream,16,0,0,0,0,0,1077,262,371,290,155
2025-01-03,instant face lift cream,38,0,0,0,0,0,1456,247,570,358,281
2025-01-04,instant face lift cream,574,60,0,20,0,40,1101,34,498,498,71
2025-01-05,instant face lift cream,188,60,40,20,0,0,977,22,404,376,174


In [630]:
adjusted_results_2= adjust_clicks_and_orders(adjusted_results)
adjusted_results_2[19].tail()

,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-27,shower curtain,55,498,431,18,24,25,4243,3371,488,199,185
2025-01-28,shower curtain,64,426,367,16,20,22,2959,2324,340,158,137
2025-01-29,shower curtain,53,633,552,18,29,34,3892,3081,423,216,172
2025-01-30,shower curtain,52,504,432,18,27,27,3860,3049,403,215,193
2025-01-31,shower curtain,62,488,421,19,23,25,5174,4074,580,282,238


In [632]:
adjusted_results_3= adjust_clicks_and_orders(adjusted_results_2)
adjusted_results_3[19].tail(10)

,Search_Term,SFR,Total_Orders,Other_Orders,ASIN1_Orders,ASIN2_Orders,ASIN3_Orders,Total_Clicks,Other_Clicks,ASIN1_Clicks,ASIN2_Clicks,ASIN3_Clicks
2025-01-22,shower curtain,64,572,497,17,32,26,4423,3434,533,249,207
2025-01-23,shower curtain,68,540,465,20,30,25,4651,3599,579,260,213
2025-01-24,shower curtain,71,730,616,32,44,38,3503,2753,413,172,165
2025-01-25,shower curtain,49,688,593,25,36,34,4422,3474,532,210,206
2025-01-26,shower curtain,32,486,416,16,30,24,3214,2534,384,153,143
2025-01-27,shower curtain,55,498,431,18,24,25,4243,3371,488,199,185
2025-01-28,shower curtain,64,426,367,16,20,22,2959,2324,340,158,137
2025-01-29,shower curtain,53,633,552,18,29,34,3892,3081,423,216,172
2025-01-30,shower curtain,52,504,432,18,27,27,3860,3049,403,215,193
2025-01-31,shower curtain,62,488,421,19,23,25,5174,4074,580,282,238


In [300]:
#Валідація даних   (для себе)

def validate_adjusted_data(adjusted_results: List[pd.DataFrame]) -> dict:
    """
    Validates the adjusted data for remaining anomalies and problematic rows.
    Returns a dictionary with validation results for each search term.
    """
    validation_results = {}
    
    for df in adjusted_results:
        search_term = df['Search_Term'].iloc[0]
        validation_results[search_term] = {
            'sfr_without_clicks': check_sfr_without_clicks(df),
            'out_of_bounds_rows': check_out_of_bounds_values(df)
        }  
    return validation_results

def check_sfr_without_clicks(df: pd.DataFrame) -> List[dict]:
    """
    Checks for rows where SFR exists but all click values are zero.
    """
    problematic_rows = []
    
    for idx in df.index:
        if pd.notna(df.at[idx, 'SFR']):  # SFR is not NaN
            click_columns = ['Total_Clicks', 'Other_Clicks', 
                           'ASIN1_Clicks', 'ASIN2_Clicks', 'ASIN3_Clicks']
            if all(df.at[idx, col] == 0 for col in click_columns):
                row_data = {
                    'date': idx,
                    'SFR': df.at[idx, 'SFR'],
                    'Search_Term': df.at[idx, 'Search_Term']
                }
                problematic_rows.append(row_data)
    return problematic_rows

def check_out_of_bounds_values(df: pd.DataFrame) -> List[dict]:
    """
    Checks for non-zero rows that are still outside the acceptable range.
    """
    problematic_rows = []
    # Calculate medians for clicks and orders
    median_clicks = get_valid_median_clicks(df['Total_Clicks'])
    median_orders = get_valid_median(df['Total_Orders'], index=2000)
    if median_clicks == 0 or median_orders == 0:
        return problematic_rows
    # Define bounds
    clicks_lower = 0.4 * median_clicks
    clicks_upper = 1.4 * median_clicks
    orders_lower = 0.4 * median_orders
    orders_upper = 1.4 * median_orders
    for idx in df.index:
        clicks = df.at[idx, 'Total_Clicks']
        orders = df.at[idx, 'Total_Orders']
        # Skip rows with all zeros
        if clicks == 0 and orders == 0:
            continue
        out_of_bounds = False
        bounds_info = {}
        if clicks > 0 and (clicks < clicks_lower or clicks > clicks_upper):
            out_of_bounds = True
            bounds_info['clicks'] = {
                'value': clicks,
                'median': median_clicks,
                'allowed_range': f"{clicks_lower:.1f} - {clicks_upper:.1f}"
            }
        if orders > 0 and (orders < orders_lower or orders > orders_upper):
            out_of_bounds = True
            bounds_info['orders'] = {
                'value': orders,
                'median': median_orders,
                'allowed_range': f"{orders_lower:.1f} - {orders_upper:.1f}"
            }
        if out_of_bounds:
            row_data = {
                'date': idx,
                'Search_Term': df.at[idx, 'Search_Term'],
                'bounds_info': bounds_info
            }
            problematic_rows.append(row_data)
    return problematic_rows

    
def print_validation_results(validation_results: dict):
    """
    Prints validation results in a readable format.
    """
    for search_term, results in validation_results.items():
        print(f"\nResults for search term: {search_term}")
        
        # Print rows with SFR but no clicks
        sfr_issues = results['sfr_without_clicks']
        if sfr_issues:
            print("\nRows with SFR but no clicks:")
            for row in sfr_issues:
                print(f"Date: {row['date']}, SFR: {row['SFR']}")
        # Print out of bounds rows
        bounds_issues = results['out_of_bounds_rows']
        if bounds_issues:
            print("\nRows with values outside acceptable range:")
            for row in bounds_issues:
                print(f"\nDate: {row['date']}")
                bounds_info = row['bounds_info']
                if 'clicks' in bounds_info:
                    click_info = bounds_info['clicks']
                    print(f"Clicks: {click_info['value']} (median: {click_info['median']}, "
                          f"allowed range: {click_info['allowed_range']})")
                if 'orders' in bounds_info:
                    order_info = bounds_info['orders']
                    print(f"Orders: {order_info['value']} (median: {order_info['median']}, "
                          f"allowed range: {order_info['allowed_range']})")
        
        if not sfr_issues and not bounds_issues:
            print("No issues found.")


In [634]:
validation_results = validate_adjusted_data(adjusted_results_3)
print_validation_results(validation_results)


Results for search term: airpods

Rows with values outside acceptable range:

Date: 2025-01-12 00:00:00
Clicks: 4909 (median: 3285.0, allowed range: 1314.0 - 4599.0)

Date: 2025-01-25 00:00:00
Clicks: 4770 (median: 3285.0, allowed range: 1314.0 - 4599.0)

Results for search term: the wild robot

Rows with values outside acceptable range:

Date: 2025-01-11 00:00:00
Clicks: 4757 (median: 3289.0, allowed range: 1315.6 - 4604.6)

Results for search term: cross
No issues found.

Results for search term: culpa mia
No issues found.

Results for search term: transformers
No issues found.

Results for search term: apple watch

Rows with values outside acceptable range:

Date: 2025-01-05 00:00:00
Clicks: 4554 (median: 3194.0, allowed range: 1277.6 - 4471.6)

Results for search term: iphone 16 pro max case

Rows with values outside acceptable range:

Date: 2025-01-04 00:00:00
Clicks: 4872 (median: 3442.0, allowed range: 1376.8 - 4818.8)

Date: 2025-01-30 00:00:00
Clicks: 4877 (median: 3442.0, al

In [635]:
# комбінуємо 100 (кількість серч термів) таблиць в одну

def create_combined_dataframe(list_of_dataframes: list) -> pd.DataFrame:
    """
    Створює DataFrame, де кожен рядок відповідає окремому DataFrame-у зі списку,
    а стовпці містять дані за всі дати.

    Args:
        list_of_dataframes: Список DataFrame-ів.

    Returns:
        Новий DataFrame.
    """

    combined_data = []
    for df in list_of_dataframes:
        row_data = {'Search_Term': df['Search_Term'].iloc[0]}  
        for date in df.index:
            row_data[f'SFR {date.date()}'] = df.loc[date, 'SFR']
            row_data[f'Orders {date.date()}'] = df.loc[date, 'Total_Orders']
            row_data[f'Clicks {date.date()}'] = df.loc[date, 'Total_Clicks']

        combined_data.append(row_data)
    combined_df = pd.DataFrame(combined_data)    
    for column in combined_df.columns:
        if 'SFR' in column:
            combined_df[column] = combined_df[column].fillna(0).astype(int)
            
    return combined_df


In [636]:
combined_df_3 = create_combined_dataframe(adjusted_results_3)
combined_df_3.head(8)

,Search_Term,SFR 2025-01-01,Orders 2025-01-01,Clicks 2025-01-01,SFR 2025-01-02,Orders 2025-01-02,Clicks 2025-01-02,SFR 2025-01-03,Orders 2025-01-03,Clicks 2025-01-03,...,Clicks 2025-01-28,SFR 2025-01-29,Orders 2025-01-29,Clicks 2025-01-29,SFR 2025-01-30,Orders 2025-01-30,Clicks 2025-01-30,SFR 2025-01-31,Orders 2025-01-31,Clicks 2025-01-31
0,airpods,20,819,3626,25,740,3333,18,833,2310,...,3333,18,588,3701,13,820,3150,25,524,4507
1,the wild robot,21,94,3333,64,103,2816,25,0,2689,...,2716,318,101,2973,217,111,3334,40,0,2798
2,cross,22,0,704,48,0,738,28,0,770,...,738,372,0,786,346,0,1146,276,0,1164
3,culpa mia,23,0,1563,31,0,1110,32,0,1214,...,1940,1999,0,1864,1245,0,1380,868,0,1300
4,transformers,24,0,857,56,0,996,41,0,1166,...,1255,158,0,1283,118,0,875,72,0,901
5,apple watch,25,571,3896,33,579,3794,31,1080,3281,...,4061,37,856,3733,34,916,3333,39,964,3333
6,iphone 16 pro max case,26,768,3280,28,688,3307,19,755,4338,...,4103,41,822,3325,38,563,4877,44,693,4821
7,glp-1,27,0,4884,6,140,6112,7,138,9200,...,4902,579,98,6168,898,135,8270,411,0,5373


In [637]:
# присвоєння середнього арифметичного * 0.7 для нульових замовлень
def process_orders(df):
    # Отримуємо всі стовпці з 'SFR' в назві
    sfr_columns = [col for col in df.columns if 'SFR' in col and col != 'Search_TermSFR']
    
    # Для кожного SFR стовпця знаходимо відповідний Orders стовпець
    for sfr_col in sfr_columns:
        # Отримуємо дату з назви стовпця SFR (припускаємо формат 'SFR YYYY-MM-DD')
        date = sfr_col.split('SFR ')[1]
        
        # Формуємо назву відповідного стовпця Orders
        orders_col = f'Orders {date}'
        
        # Перевіряємо чи існує такий стовпець Orders
        if orders_col in df.columns:
            # Отримуємо всі стовпці Orders, крім поточного
            all_orders_columns = [col for col in df.columns if 'Orders' in col and col != orders_col]
            
            # Для кожного рядка
            for idx in df.index:
                # Якщо SFR не 0, а Orders 0
                if df.at[idx, sfr_col] != 0 and df.at[idx, orders_col] == 0:
                    # Рахуємо середнє по всіх інших Orders для цього рядка
                    other_orders = df.loc[idx, all_orders_columns]
                    non_zero_orders = other_orders[other_orders != 0]
                    
                    # Рахуємо середнє по ненульових Orders, якщо такі є
                    if len(non_zero_orders) > 0:
                        mean_orders = non_zero_orders.mean()
                        # Присвоюємо нове значення
                        df.at[idx, orders_col] = round(mean_orders * 0.7)
    return df


In [638]:
combined_df_3 = process_orders(combined_df_3)
combined_df_3.head(8)

,Search_Term,SFR 2025-01-01,Orders 2025-01-01,Clicks 2025-01-01,SFR 2025-01-02,Orders 2025-01-02,Clicks 2025-01-02,SFR 2025-01-03,Orders 2025-01-03,Clicks 2025-01-03,...,Clicks 2025-01-28,SFR 2025-01-29,Orders 2025-01-29,Clicks 2025-01-29,SFR 2025-01-30,Orders 2025-01-30,Clicks 2025-01-30,SFR 2025-01-31,Orders 2025-01-31,Clicks 2025-01-31
0,airpods,20,819,3626,25,740,3333,18,833,2310,...,3333,18,588,3701,13,820,3150,25,524,4507
1,the wild robot,21,94,3333,64,103,2816,25,67,2689,...,2716,318,101,2973,217,111,3334,40,62,2798
2,cross,22,0,704,48,0,738,28,0,770,...,738,372,0,786,346,0,1146,276,0,1164
3,culpa mia,23,0,1563,31,0,1110,32,0,1214,...,1940,1999,0,1864,1245,0,1380,868,0,1300
4,transformers,24,0,857,56,0,996,41,0,1166,...,1255,158,0,1283,118,0,875,72,0,901
5,apple watch,25,571,3896,33,579,3794,31,1080,3281,...,4061,37,856,3733,34,916,3333,39,964,3333
6,iphone 16 pro max case,26,768,3280,28,688,3307,19,755,4338,...,4103,41,822,3325,38,563,4877,44,693,4821
7,glp-1,27,86,4884,6,140,6112,7,138,9200,...,4902,579,98,6168,898,135,8270,411,85,5373


In [639]:
# Підраховуємо кількість рядків, де всі замовлення  дорівнюють 0
rows_with_zero = combined_df_3[(combined_df_3 == 0).any(axis=1)]

# Виводимо кількість таких рядків
rows_with_zero.shape


(23, 94)

In [479]:
combined_df.dtypes

Search_Term             object
SFR 2025-01-01           int64
Orders 2025-01-01        int64
Clicks 2025-01-01        int64
SFR 2025-01-02           int64
                        ...   
Orders 2025-01-31        int64
Clicks 2025-01-31        int64
magnet weekly sales    float64
Monthly Orders           int64
Monthly Clicks           int64
Length: 97, dtype: object

In [455]:
#вибираємо всі серч терми для Hellium
search = combined_df['Search_Term'].to_list()

for i in range(len(search)): 
    print(search[i])

airpods
the wild robot
cross
culpa mia
transformers
apple watch
iphone 16 pro max case
glp-1
ipad 10th generation case
nad supplement
lioness
urolithin a
ornament storage box
harry potter movies set 1-8 on prime video
walking pad
the substance
vanity
magnesium glycinate
eggs
shower curtain
iphone 16 pro case
the chosen
the equalizer
air fryer
aa batteries
shoe rack
nintendo switch
lego
sonic
paw patrol
a quite place
the agency
desk
ps5 controller
protein powder
owala
pheromone cologne for men
sonic the hedgehog 3
onyx storm
gaming chair
gladiator ii
ps5
fire tv stick
toothpaste
the last days of ptolemy grey
juror #2
hair perfume
coffee maker
aaa batteries
shower head
methylene blue
heated blanket
apple watch bands for women
let them book mel robbins
digital camera
tulsa king
dresser for bedroom
from
candle warmer lamp
apple pencil
2025 calendar
laptop
gift cards
air purifier
alexa
queen bed frame
kindle paperwhite 2024
sonic the hedgehog 2
reacher
creatine
smile 2
venom
ipad 10th gener

In [457]:
import warnings

# дані з Hellium

warnings.simplefilter(action='ignore', category=UserWarning)
file_path = r'C:\Users\user\Downloads\US_AMAZON_magnet__2025-02-19.xlsx'
df_magnet = pd.read_excel(file_path, usecols=['Keyword Phrase', 'Keyword Sales'])

# Переглянути результат
df_magnet.head()


,Keyword Phrase,Keyword Sales
0,christmas tree bag,1021
1,ornament storage box,1409
2,the last days of ptolemy grey,112
3,2025 calendar,1312
4,wall calendar 2025,1562


In [458]:
df_magnet.tail()

,Keyword Phrase,Keyword Sales
94,reacher,2105
95,airpods,4445
96,eggs,9805
97,nad supplement,4000
98,urolithin a,1949


In [640]:
# Заповнюємо стовпець magnet weekly sales

# Створити словник для швидкого пошуку значень 'Keyword Sales' за 'Keyword Phrase'
sales_dict = df_magnet.set_index('Keyword Phrase')['Keyword Sales'].to_dict()

# Створити новий стовпець 'magnet weekly sales' у combined_df
combined_df_3['magnet weekly sales'] = combined_df_3['Search_Term'].map(sales_dict).fillna(0)

# Переглянути результат
combined_df_3.tail()


,Search_Term,SFR 2025-01-01,Orders 2025-01-01,Clicks 2025-01-01,SFR 2025-01-02,Orders 2025-01-02,Clicks 2025-01-02,SFR 2025-01-03,Orders 2025-01-03,Clicks 2025-01-03,...,SFR 2025-01-29,Orders 2025-01-29,Clicks 2025-01-29,SFR 2025-01-30,Orders 2025-01-30,Clicks 2025-01-30,SFR 2025-01-31,Orders 2025-01-31,Clicks 2025-01-31,magnet weekly sales
96,iphone charger,116,363,2470,118,520,4634,121,413,4474,...,116,414,3470,121,555,2901,161,432,3565,2000.0
97,gel nail polish,117,566,4090,135,506,4495,112,550,3277,...,167,558,4708,162,649,4638,165,608,4572,2580.0
98,wall calendar 2025,118,612,2424,88,401,3333,146,570,2386,...,2034,635,2864,2285,605,1856,2636,552,2779,1562.0
99,secret level,119,0,1122,140,0,902,78,0,884,...,380,0,616,348,0,621,295,0,747,0.0
100,christmas tree bag,120,464,2221,188,490,1336,244,410,1564,...,15787,343,1449,16860,320,1548,16906,380,1484,1021.0


In [641]:
# Загальні замовлення та кліки

orders_columns = [col for col in combined_df_3.columns if 'Orders' in col]

# Створити новий стовпець "Monthly Orders" як суму значень стовпців "Orders" в кожному рядку
combined_df_3['Monthly Orders'] = combined_df_3[orders_columns].sum(axis=1)

# Створити список стовпців, які містять "Clicks"
clicks_columns = [col for col in combined_df_3.columns if 'Clicks' in col]

# Створити новий стовпець "Monthly Clicks" як суму значень стовпців "Clicks" в кожному рядку
combined_df_3['Monthly Clicks'] = combined_df_3[clicks_columns].sum(axis=1)

# Переглянути результат
combined_df_3[['magnet weekly sales', 'Monthly Orders', 'Monthly Clicks']].tail(20)


,magnet weekly sales,Monthly Orders,Monthly Clicks
81,877.0,1173,68730
82,1654.0,20007,110036
83,4059.0,1512,32545
84,1337.0,0,52850
85,2275.0,15113,102638
86,3683.0,13136,98677
87,1433.0,19070,97677
88,2490.0,20943,82889
89,3727.0,15984,88886
90,2535.0,15385,82487


In [642]:
combined_df_3.iloc[77:82]

,Search_Term,SFR 2025-01-01,Orders 2025-01-01,Clicks 2025-01-01,SFR 2025-01-02,Orders 2025-01-02,Clicks 2025-01-02,SFR 2025-01-03,Orders 2025-01-03,Clicks 2025-01-03,...,Clicks 2025-01-29,SFR 2025-01-30,Orders 2025-01-30,Clicks 2025-01-30,SFR 2025-01-31,Orders 2025-01-31,Clicks 2025-01-31,magnet weekly sales,Monthly Orders,Monthly Clicks
77,gonggi,97,566,1606,90,582,2951,147,462,2991,...,1792,539,628,2886,556,642,2690,1453.0,16246,68102
78,mouth tape for sleeping,98,722,3890,137,525,4139,141,547,2251,...,2500,274,489,3026,338,675,3078,6024.0,17434,98379
79,valentines day decorations,99,975,4856,98,684,3234,83,999,4752,...,3097,63,873,3939,73,946,5216,4354.0,22609,129015
80,vitamin d,100,634,5146,70,526,2852,74,774,4194,...,5000,74,945,2976,107,588,4097,3473.0,21170,129714
81,conclave,101,45,1643,362,43,1716,209,41,2848,...,3040,488,69,2985,155,27,2856,877.0,1173,68730


In [643]:
# зберігаємо в excel
combined_df_3.to_excel("combined_data_100_tweaked.xlsx", index=False)

In [326]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment

# Збереження DataFrame у файл Excel
#combined_df.to_excel("combined_data_puzzles.xlsx", index=False)

# Завантаження файлу Excel для редагування
workbook = load_workbook("combined_data_puzzles.xlsx")
sheet = workbook.active

# Застосування форматування заголовків стовпців
for cell in sheet[1]:
    cell.alignment = Alignment(text_rotation=90, wrap_text=True, horizontal='center', vertical='center')

# Збереження змін
workbook.save("combined_data_puzzles.xlsx")


In [88]:

def export_to_excel(processed_results: List[pd.DataFrame], output_path: str):
    """
    Export all DataFrames to Excel with each DataFrame on its own sheet,
    named after its Search_Term value.
    """
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for df in processed_results:
            # Get sheet name from the first Search_Term value
            sheet_name = str(df['Search_Term'].iloc[0])
            # Excel has a 31 character limit for sheet names
            if len(sheet_name) > 30:
                sheet_name = sheet_name[:30]
            
            # Export to Excel
            df.to_excel(writer, sheet_name=sheet_name, index=True)
            
            # Get workbook and worksheet objects
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]
            
            # Auto-adjust columns width
            for idx, col in enumerate(df.columns):
                series = df[col]
                max_len = max(
                    series.astype(str).apply(len).max(),  # len of largest item
                    len(str(series.name))  # len of column name/header
                ) + 1  # adding a little extra space
                worksheet.set_column(idx + 1, idx + 1, max_len)  # set column width
            
            # Set the width of the index column
            worksheet.set_column(0, 0, 15)  # Adjust width for datetime index



In [91]:
# Після всіх попередніх кроків
processed_results = process_nan_rows(adjusted_results)
export_to_excel(processed_results, 'search_terms_analysis.xlsx')